# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 12:58PM,243889,19,ADV80004798,"AdvaGen Pharma, Ltd",Released
1,Aug 10 2022 12:57PM,243888,19,60016519,"GUSA Granules USA, Inc.",Released
2,Aug 10 2022 12:55PM,243887,10,DNMD-23521-BO,Emerginnova,Released
3,Aug 10 2022 12:55PM,243887,10,DNMD-23537-BO,Emerginnova,Released
4,Aug 10 2022 12:55PM,243887,10,DNMD-23589-BO,Emerginnova,Released
5,Aug 10 2022 12:55PM,243887,10,Enova-10455,Emerginnova,Released
6,Aug 10 2022 12:55PM,243887,10,Enova-10456,Emerginnova,Released
7,Aug 10 2022 12:55PM,243887,10,Enova-10457,Emerginnova,Released
8,Aug 10 2022 12:55PM,243887,10,Enova-10458,Emerginnova,Released
9,Aug 10 2022 12:26PM,243885,12,6826069,Doctors Scientific Organica,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,243883,Released,5
31,243885,Released,1
32,243887,Released,7
33,243888,Released,1
34,243889,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243883,NaN,NaN,5.0
243885,NaN,NaN,1.0
243887,NaN,NaN,7.0
243888,NaN,NaN,1.0
243889,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243769,0.0,0.0,9.0
243805,0.0,0.0,15.0
243808,32.0,8.0,15.0
243811,0.0,0.0,3.0
243828,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243769,0,0,9
243805,0,0,15
243808,32,8,15
243811,0,0,3
243828,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243769,0,0,9
1,243805,0,0,15
2,243808,32,8,15
3,243811,0,0,3
4,243828,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243769,,,9
1,243805,,,15
2,243808,32,8,15
3,243811,,,3
4,243828,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 12:58PM,243889,19,"AdvaGen Pharma, Ltd"
1,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc."
2,Aug 10 2022 12:55PM,243887,10,Emerginnova
9,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica
10,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation
15,Aug 10 2022 12:20PM,243881,19,ACG North America LLC
16,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation
17,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc."
19,Aug 10 2022 12:19PM,243879,19,ACG North America LLC
20,Aug 10 2022 12:18PM,243878,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 12:58PM,243889,19,"AdvaGen Pharma, Ltd",,,1
1,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",,,1
2,Aug 10 2022 12:55PM,243887,10,Emerginnova,,,7
3,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,,,1
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,,,5
5,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,,,1
6,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,,,1
7,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",,,2
8,Aug 10 2022 12:19PM,243879,19,ACG North America LLC,,,1
9,Aug 10 2022 12:18PM,243878,10,Bio-PRF,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:58PM,243889,19,"AdvaGen Pharma, Ltd",1,,
1,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1,,
2,Aug 10 2022 12:55PM,243887,10,Emerginnova,7,,
3,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,1,,
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5,,
5,Aug 10 2022 12:20PM,243881,19,ACG North America LLC,1,,
6,Aug 10 2022 12:20PM,243882,10,ISDIN Corporation,1,,
7,Aug 10 2022 12:20PM,243880,19,"GUSA Granules USA, Inc.",2,,
8,Aug 10 2022 12:19PM,243879,19,ACG North America LLC,1,,
9,Aug 10 2022 12:18PM,243878,10,Bio-PRF,3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:58PM,243889,19,"AdvaGen Pharma, Ltd",1,,
1,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1,,
2,Aug 10 2022 12:55PM,243887,10,Emerginnova,7,,
3,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,1,,
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:58PM,243889,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Aug 10 2022 12:55PM,243887,10,Emerginnova,7.0,NaN,NaN
3,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,1.0,NaN,NaN
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:58PM,243889,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Aug 10 2022 12:55PM,243887,10,Emerginnova,7.0,0.0,0.0
3,Aug 10 2022 12:26PM,243885,12,Doctors Scientific Organica,1.0,0.0,0.0
4,Aug 10 2022 12:21PM,243883,10,ISDIN Corporation,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1950925,51.0,17.0,0.0
12,243885,1.0,0.0,0.0
15,975449,45.0,20.0,0.0
16,243865,1.0,0.0,0.0
19,1707167,7.0,1.0,0.0
20,1706789,77.0,17.0,32.0
21,243828,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1950925,51.0,17.0,0.0
1,12,243885,1.0,0.0,0.0
2,15,975449,45.0,20.0,0.0
3,16,243865,1.0,0.0,0.0
4,19,1707167,7.0,1.0,0.0
5,20,1706789,77.0,17.0,32.0
6,21,243828,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,51.0,17.0,0.0
1,12,1.0,0.0,0.0
2,15,45.0,20.0,0.0
3,16,1.0,0.0,0.0
4,19,7.0,1.0,0.0
5,20,77.0,17.0,32.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,51.0
1,12,Released,1.0
2,15,Released,45.0
3,16,Released,1.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,32.0,0.0
Executing,17.0,0.0,20.0,0.0,1.0,17.0,1.0
Released,51.0,1.0,45.0,1.0,7.0,77.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,32.0,0.0
1,Executing,17.0,0.0,20.0,0.0,1.0,17.0,1.0
2,Released,51.0,1.0,45.0,1.0,7.0,77.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,32.0,0.0
1,Executing,17.0,0.0,20.0,0.0,1.0,17.0,1.0
2,Released,51.0,1.0,45.0,1.0,7.0,77.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()